In [ ]:
from Beamformer_MVDR import beamformer_MVDR
import utils
from acoular import MicGeom
import numpy as np

# Parameters
H5FILENAME = '/media/xian/Data/dataset/Bekaert/h5files/42.h5'
OUTPUT_FILENAME = 'enhanced_audio.wav'

mg = MicGeom(from_file='array_9.xml')
number_of_mic = mg.mpos.shape[1]
MIC_POS = []
for i in np.arange(number_of_mic):
    MIC_POS.append(mg.mpos[:,int(i)])

LOOK_POS = [4, 2, 0.5]
SAMPLING_RATE = 51200
FFT_LENGTH = 512
FFT_SHIFT = 256
SOUND_SPEED = 343

In [ ]:
multi_signal = utils.get_data_from_h5(H5FILENAME, skip_last_channel=True)

mvdr_beamformer = beamformer_MVDR(MIC_POS, sampling_rate=SAMPLING_RATE, fft_length=FFT_LENGTH, fft_shift=FFT_SHIFT, sound_speed=SOUND_SPEED)

steering_vector = mvdr_beamformer.get_steering_vector_near_field(LOOK_POS)

spatial_correlation_matrix = mvdr_beamformer.get_spatial_correlation_matrix(multi_signal)

beamformer = mvdr_beamformer.get_mvdr_beamformer(steering_vector, spatial_correlation_matrix)

complex_spectrum = utils.get_spectrogram(multi_signal, FFT_LENGTH, FFT_LENGTH, FFT_LENGTH) # no overlap, no window, for ifft

enhanced_spectrum = mvdr_beamformer.apply_beamformer(beamformer, complex_spectrum)

In [ ]:
from scipy.io.wavfile import write

enhanced_audio = utils.spec2wav(enhanced_spectrum, SAMPLING_RATE, FFT_LENGTH, FFT_LENGTH, FFT_LENGTH)

enhanced_audio = enhanced_audio / np.max(np.abs(enhanced_audio)) * 0.7

write(OUTPUT_FILENAME, SAMPLING_RATE, enhanced_audio.astype(np.float32))